In [1]:
from osgeo import gdal
from osgeo import ogr

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import netCDF4 as nc

# converting the datetime format
from datetime import datetime

In [2]:
# DATE time function
def conv_date(d):
    return datetime.strptime(str(d), '%Y%m%d%H%M%S%f')

In [12]:
# FUNCTION to convert data
def convHdf(path_file, n=0):
    city_lat,city_lon = 31.57,74.40

    data= nc.Dataset(path_file)

    # get the HDF data and convert to CSV
    df_xco2= pd.DataFrame()

    df_xco2['Xco2']= data.variables['xco2'][:]
    df_xco2['Latitude']= data.variables['latitude'][:]
    df_xco2['Longitude']= data.variables['longitude'][:] 
    df_xco2['quality_flag']= data.variables['xco2_quality_flag'][:] 
    
    # Date
    df_xco2['DateTime']= data.variables['sounding_id'][:]
    
    #Convert soundingID to datetime format
    df_xco2['DateTime']= df_xco2['DateTime'].apply(conv_date)
    df_xco2['DateTime']= pd.to_datetime(df_xco2['DateTime'])
    
    # YEAR and month column
    df_xco2['Year']= df_xco2['DateTime'].dt.year
    df_xco2['Month']= df_xco2['DateTime'].dt.month
    df_xco2['Day']= df_xco2['DateTime'].dt.day
    
    # Refine the ENTIRE dataframe by GOOD quality_flag->0
    # NOTE: REDUCES the size of the file
    df_xco2= df_xco2[df_xco2['quality_flag'] == 0]   
    
   
    date= str(data.variables['sounding_id'][0])      
    
    # create a CSV and store on new folder: csv_files
    # df_xco2.to_csv('oco2_xco2_'+ date+'_.csv', index= False)

    lat = df_xco2['Latitude']
    lon = df_xco2['Longitude']

    sq_diff_lat = (lat - city_lat)**2
    sq_diff_lon = (lon - city_lon)**2

    # Compute the index of the lat lon
    min_index_lat = sq_diff_lat.argmin()
    min_index_lon = sq_diff_lon.argmin()

    print(min_index_lat,min_index_lon)
    print(df_xco2[min_index_lat])

In [10]:
path_file= './CO2_Files/oco2_LtCO2_140906_B11100Ar_230523232559s.nc4'

In [9]:
# os.chdir('./XCO')
os.getcwd()

'f:\\Subhan_Ali_Kazmi\\notebooks\\XCO'

In [11]:
# call the function and pass the file path
convHdf(path_file)

In [6]:
df_2019= pd.read_csv('oco2_xco2_2014090602072531_.csv')

In [14]:
df_2019.head(4)


,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day
0,392.70150,54.485320,160.50188,0,2014-09-06 02:07:25.710,2014,9,6
1,393.74164,54.482136,160.53796,0,2014-09-06 02:07:25.730,2014,9,6
2,394.21582,54.503334,160.51003,0,2014-09-06 02:07:26.020,2014,9,6
3,394.28003,54.501705,160.52809,0,2014-09-06 02:07:26.030,2014,9,6


In [7]:
if not os.path.exists('json_format'):
    os.makedirs('json_format')
else:
    print("Directory exists!")

In [8]:
# !ogr2ogr -oo X_POSSIBLE_NAMES=Longitude -oo Y_POSSIBLE_NAMES=Latitude -a_srs "EPSG:4326" json_format/oco2_2020.json oco2_xco2_2014090602072531_.csv

from osgeo import gdal
gdal.VectorTranslate('json_format/oco2_2014.json', 'oco2_xco2_2014090602072531_.csv', options='-oo X_POSSIBLE_NAMES=Longitude -oo Y_POSSIBLE_NAMES=Latitude -a_srs "EPSG:4326"')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000027C72404540> >

In [11]:
!mkdir tif_format

In [26]:
# !gdal_rasterize -a Xco2 -a_nodata 0 -ts 712 356 json_format/oco2_2020.json tif_format/oco2_2020.tif

from osgeo import gdal
options = gdal.RasterizeOptions(options='-a Xco2 -a_nodata 0 -ts 512 256')
gdal.Rasterize('tif_format/oco2_2020.tif','json_format/oco2_2014.json',  options=options)


In [ ]:
!dir

In [ ]:
if not os.path.exists('color_tif_file'):
    os.makedirs('color_tif_file')
else:
    print("Directory exists!")

In [19]:
# !gdaldem color-relief -alpha tif_format/oco2_2020.tif colormap.txt color_tif_file/oco2_2020_color.tif
from osgeo import gdal
gdal.DEMProcessing('color_tif_file/oco2_2014_color.tif', 'tif_format/oco2_2020.tif', 'color-relief', colorFilename='colormap.txt', addAlpha=True)

ValueError: Received a NULL pointer.

In [ ]:
if not os.path.exists('color_PNG_FILE'):
    os.makedirs('color_PNG_FILE')
else:
    print("Directory exists")

In [ ]:
!gdal_translate -of PNG -co "WORLDFILE=YES" color_tif_file/oco2_2020_color.tif color_PNG_FILE/oco2_2020_PNG.png

from osgeo import gdal
gdal.Translate('color_PNG_FILE/oco2_2020_PNG.png', 'color_tif_file/oco2_2020_color.tif', format='PNG', creationOptions=['WORLDFILE=YES'])

In [ ]:
dataset_a= gdal.Open('color_tif_file/oco2_2020_color.tif', gdal.GA_ReadOnly)


In [ ]:
band= dataset_a.GetRasterBand(4)

arr= band.ReadAsArray()
plt.imshow(arr)